![Image](actinia_logo.png)

## User defined processing

### The actinia process chain

Actinia provides the **process chain** approach to specify import,
processing and export of geodata using the actinia GRASS GIS processing
system. The process chain must be formulated in JSON. The processing is
always performed in an ephemeral database. The computational environment
is based on locations in the persistent database. If required, the
ephemeral database can be moved into the persistent user database, so
that the computational results can be used in further processing steps
or visualized using the actinia rendering REST calls.

The ephemeral database will be removed after computation. However, all
raster and vector data that was generated during the processing can be
exported using GDAL/OGR specific datatypes and stored in an object
storage, outside the actinia environment. Within a process chain we have
read only access to all raster maps of the persistent database location
that is used as computational environment.

<!---
no longer generated: https://actinia.mundialis.de/api_docs
--->
A process chain is a list of [GRASS GIS modules](https://grass.osgeo.org/grass-stable/manuals/index.html) that will be executed in serial, based on the order of the list. GRASS GIS modules are specified as [process definitions](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/latest/swagger.json#tag/Processing) that include the name of the command, the inputs and outputs, including import and export definitions as well as the module flags.

This tutorial demonstrates the actinia REST service access using the Python
library **requests**.

The following example is a Jupyter Notebook version of the online [actinia tutorial](https://actinia-dev.mundialis.de/tutorial/tutorial_process_chain.html). 

---

### actinia API documentation

* [Stable actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [Development actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Requirements

#### Software & Modules

This tutorial assumes your are comfortable with the [Python](https://python.org) programming language. Familiarity with basic REST API concepts and usage is also assumed.

Python modules used in this tutorial are:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)


#### ACTINIA API user and password

This demo requires credentials for authentication set below in **Preparation** as a variable. Another actinia instance might require different credentials.

### Helper Modules and Functions
Before interacting with the actinia server using Python, we will import required packages an set up a helper function to print formatted JSON using json.

***Note:*** *You may need to install two helpful browser plugins called **RESTman** and **JSON Formatter** that format JSON and makes it easier to read:*

* [RESTman extension](https://chrome.google.com/webstore/detail/restman/ihgpcfpkpmdcghlnaofdmjkoemnlijdi)
* [JSON Formatter](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa)

## Preparation


In [ ]:
# first, let's import the required packages.

from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth


To simplify our life in terms of server communication we store the credentials and REST server URL in  variables.

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_url = actinia_baseurl + "/api/" + actinia_version
actinia_auth = HTTPBasicAuth('demouser', 'gu3st!pa55w0rd')

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Examples

The following example defines a single process
that runs the GRASS GIS module [*r.slope.aspect*](https://grass.osgeo.org/grass-stable/manuals/r.slope.aspect.html) to compute the
*slope* for the raster map layer *elev_ned_30m* that is located in the
[mapset](https://grass.osgeo.org/grass-stable/manuals/grass_database.html) *PERMANENT*. The output of the module is named
*elev_ned_30m_slope* and should be exported as a GeoTiff file.

The corresponding process chain item as JSON:

```json
 {
   "module": "r.slope.aspect",
   "id": "r_slope_aspect_1",
   "inputs": [
     {
       "param": "elevation",
       "value": "elev_ned_30m@PERMANENT"
     }
   ],
   "outputs": [
     {
       "export": {
         "format": "GTiff",
         "type": "raster"
       },
       "param": "slope",
       "value": "elev_ned_30m_slope"
     }
   ],
   "flags": "a"
 }
```


**Import and export**

The actinia process chain supports the specification of URL\'s to raster
layers in the input definition. The following process chain imports a
raster map layer that is located in an object storage with the name
*elev_ned_30m_new* and sets the computational region for the
following processing step with the GRASS GIS module [*g.region*](https://grass.osgeo.org/grass-stable/manuals/g.region.html). Then
slope and aspect are computed with *r.slope.aspect* and specified for
export as GeoTiff files.

The corresponding process chain as JSON:


```json
 {
   "list": [
     {
       "module": "g.region",
       "id": "g_region_1",
       "inputs": [
         {
           "import_descr": {
             "source": "https://storage.googleapis.com/graas-geodata/elev_ned_30m.tif",
             "type": "raster"
           },
           "param": "raster",
           "value": "elev_ned_30m_new"
         }
       ],
       "flags": "p"
     },
     {
       "module": "r.slope.aspect",
       "id": "r_slope_aspect_1",
       "inputs": [
         {
           "param": "elevation",
           "value": "elev_ned_30m_new"
         }
       ],
       "outputs": [
         {
           "export": {
             "format": "GTiff",
             "type": "raster"
           },
           "param": "slope",
           "value": "elev_ned_30m_new_slope"
         }
       ],
       "flags": "a"
     }
   ],
   "version": "1"
 }
```

**Output parsing**

Many GRASS GIS modules produce textual output in form of lists, tables
and key/value pairs. Actinia supports the analysis of this data and the
parsing and transformation in JSON accessible data. For each GRASS GIS
module the property \"stdout\" can be specified to transform the output
of the module into a list of values, a key/value list or a table.

The following options are available as format:

 -   \"kv\" parses the module output and creates key/value pairs
 -   \"list\" parses the module output and creates a list of values
 -   \"table\" parses the module output and creates a list of lists
     with values aka 2D array aka table

Additionally the unique id that will be used in the response process
result dictionary and the delimiter must be defined.

The following process chain demonstrates all available approaches, to
parse the stdout output of a module:

```json
 {
     "version": 1,
     "list": [
         {
             "id": "1",
             "module": "g.region",
             "inputs": [
                 {"param": "raster",
                  "value": "elevation@PERMANENT"},
                 {"param": "res",
                  "value": "5000"}
             ],
             "stdout": {"id": "region", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
         {
             "id": "2",
             "module": "r.out.ascii",
             "inputs": [{"param": "input",
                         "value": "elevation@PERMANENT"},
                        {"param": "precision", "value": "0"}],
             "stdout": {"id": "elevation", "format": "table", "delimiter": " "},
             "flags": "h"
         },
         {
             "id": "3",
             "module": "g.list",
             "inputs": [{"param": "type",
                         "value": "raster"}],
             "stdout": {"id": "map_list", "format": "list", "delimiter": "\n"}
         },
         {
             "id": "4",
             "module": "r.univar",
             "inputs": [{"param": "map",
                         "value": "elevation@PERMANENT"}],
             "stdout": {"id": "stats", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
         {
             "id": "5",
             "module": "r.univar",
             "inputs": [{"param": "map",
                         "value": "elevation@PERMANENT"},
                        {"param": "zones",
                         "value": "basin_50K@PERMANENT"},
                        {"param": "separator",
                         "value": "pipe"}],
             "stdout": {"id": "stats_zonal", "format": "table", "delimiter": "|"},
             "flags": "t"
         }
     ]
 }
```

Let's send this process chain to actinia: 

In [ ]:
# make a POST request to the actinia data API
request_url = actinia_url + "/locations/nc_spm_08/processing_async"
print("actinia POST request:")
print(request_url)
print("---")

process_chain = {
     "version": 1,
     "list": [
         {
             "id": "1",
             "module": "g.region",
             "inputs": [
                 {"param": "raster",
                  "value": "elevation@PERMANENT"},
                 {"param": "res",
                  "value": "5000"}
             ],
             "stdout": {"id": "region", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
         {
             "id": "2",
             "module": "r.out.ascii",
             "inputs": [{"param": "input",
                         "value": "elevation@PERMANENT"},
                        {"param": "precision", "value": "0"}],
             "stdout": {"id": "elevation", "format": "table", "delimiter": " "},
             "flags": "h"
         },
         {
             "id": "3",
             "module": "g.list",
             "inputs": [{"param": "type",
                         "value": "raster"}],
             "stdout": {"id": "map_list", "format": "list", "delimiter": "\n"}
         },
         {
             "id": "4",
             "module": "r.univar",
             "inputs": [{"param": "map",
                         "value": "elevation@PERMANENT"}],
             "stdout": {"id": "stats", "format": "kv", "delimiter": "="},
             "flags": "g"
         },
         {
             "id": "5",
             "module": "r.univar",
             "inputs": [{"param": "map",
                         "value": "elevation@PERMANENT"},
                        {"param": "zones",
                         "value": "basin_50K@PERMANENT"},
                        {"param": "separator",
                         "value": "pipe"}],
             "stdout": {"id": "stats_zonal", "format": "table", "delimiter": "|"},
             "flags": "t"
         }
     ]
 }

request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

The response provides the status URL that must be polled to receive the
finished response:

In [ ]:
# get a json-encoded content of the response
jsonResponse = request.json()

print("Response with status code %d:" % request.status_code)

# print formatted JSON
print_as_json(jsonResponse)

Poll the status of the asynchronous API call by polling the status URL.
Be aware that you have to use the current status url as the resource id will change for different API
calls.

In [ ]:
# make a GET request to the actinia data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

Poll until there is a message **"Processing successfully finished"**

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

The result of the process chain evaluation is the following JSON
response.

The result of the stdout output parsing for each module is located in
the "process_results" section of the json response.

In [ ]:
# print formatted JSON
print_as_json(jsonResponse)

### Creating a process chain step-by-step

First, create an empty process chain

In [ ]:
# create a POST request to the Actinia Data API
request_url = actinia_url + "/locations/nc_spm_08/processing_async"

process_chain = {"version": 1, "list": []}

Set up a helper function to create list items 

In [ ]:
# function to create a process chain item

def create_actinia_pc_item(id, module,
                           inputs=None, outputs=None, flags=None, stdin=None, stdout=None,
                           overwrite=False, superquiet=False, verbose=False, interface_description=False):
    """
    Creates a list item for an actinia process chain
    
    Parameters
    ----------
    id: str
        unique id for this item
    module: str
        some valid GRASS or actinia module
    inputs: list or dict
        list of input parameters with values in the form
        [{"param": key1, "value": value1}, {"param": key2, "value": value2}, ...]
        shorter alternative as dict
        {"key1": value1, "key2": value2, ...}
    outputs: list or dict
        list of output parameters with values in the form
        [{"param": key1, "value": value1}, {"param": key2, "value": value2}, ...]
        shorter alternative as dict
        {"key1": value1, "key2": value2, ...}
    flags: str
        optional flags for the module
    stdin: dict
        options to read stdin
    stdout: dict
        options to write to stdout
        must be of the form
        {"id": value1, "format": value2, "delimiter": value3}
    overwrite: bool
        optional, set to True to allow overwriting existing data
    superquiet: bool
        optional, set to True to suppress all messages but errors
    verbose: bool
        optional, set to True to allow verbose messages
    interface_description: bool
        optional, set to True to create an interface_description
    """
    pc_item = {"id": id, "module": module}
    if inputs:
        if isinstance(inputs, list):
            pc_item["inputs"] = inputs
        elif isinstance(inputs, dict):
            tmplist = []
            for k, v in inputs.items():
                tmplist.append({"param": k, "value": v})
            pc_item["inputs"] = tmplist
    if outputs:
        if isinstance(outputs, list):
            pc_item["inputs"] = inputs
        elif isinstance(outputs, dict):
            tmplist = []
            for k, v in outputs.items():
                tmplist.append({"param": k, "value": v})
            pc_item["outputs"] = tmplist
    if flags:
        pc_item["flags"] = flags
    if stdin:
        pc_item["stdin"] = stdin
    if stdout:
        pc_item["stdout"] = stdout
    if overwrite is True:
        pc_item["overwrite"] = True
    if superquiet is True:
        pc_item["superquiet"] = True
    if verbose is True:
        pc_item["verbose"] = True
    if interface_description is True:
        pc_item["interface_description"] = True

    return pc_item
    

Add items to the process chain list

In [ ]:
# list item for g.region
list_id = 1
# long form, as in a process chain
#inputs = [{"param": "raster",
#           "value": "elevation@PERMANENT"},
#          {"param": "res",
#           "value": "5000"}
#         ]

# short form, accepted by create_actinia_pc_item
inputs = {"raster": "elevation@PERMANENT",
          "res": "5000"}

stdout = {"id": "region", "format": "kv", "delimiter": "="}
flags = "g"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="g.region",
                                 inputs=inputs,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

In [ ]:
# list item for r.out.ascii
list_id += 1
# long form
inputs = [{"param": "input",
           "value": "elevation@PERMANENT"},
          {"param": "precision", "value": "0"}]

# short form
inputs = {"input": "elevation@PERMANENT",
          "precision": "0"}

stdout = {"id": "elevation", "format": "table", "delimiter": " "}
flags = "h"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.out.ascii",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

In [ ]:
# list item for g.list
list_id += 1
# long form
inputs = [{"param": "type",
           "value": "raster"}]
# short form
inputs = {"type": "raster"}

stdout = {"id": "map_list", "format": "list", "delimiter": "\n"}
pc_item = create_actinia_pc_item(id=list_id,
                                 module="g.list",
                                 inputs=inputs,
                                 stdout=stdout)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

In [ ]:
# list item for r.univar (key-value output)
list_id += 1
# long form
inputs = [{"param": "map",
           "value": "elevation@PERMANENT"}]
# short form
inputs = {"map": "elevation@PERMANENT"}

stdout = {"id": "stats", "format": "kv", "delimiter": "="}
flags = "g"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.univar",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

In [ ]:
# list item for r.univar (table output)
list_id += 1
# long form
inputs = [{"param": "map",
           "value": "elevation@PERMANENT"},
          {"param": "zones",
                    "value": "basin_50K@PERMANENT"},
          {"param": "separator",
           "value": "pipe"}]
# short form
inputs = {"map": "elevation@PERMANENT",
          "zones": "basin_50K@PERMANENT",
          "separator": "pipe"}

stdout = {"id": "stats_zonal", "format": "table", "delimiter": "|"}
flags = "t"
pc_item = create_actinia_pc_item(id=list_id,
                                 module="r.univar",
                                 inputs=inputs,
                                 stdout=stdout,
                                 flags=flags)
process_chain["list"].append(pc_item)

print_as_json(process_chain)

In [ ]:
# submit the job
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

In [ ]:
# get a json-encoded content of the response
jsonResponse = request.json()

print("Response with status code %d:" % request.status_code)

# print formatted JSON
print_as_json(jsonResponse)

In [ ]:
# make a GET request to the Actinia Data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

### Sentinel-2A NDVI process chain

We create a process chain that computes the NDVI from a Sentinel-2A
scene based on the bands 8 and 4 with the GRASS GIS module *r.mapcalc*. We
use the North Carolina sample location **nc\_spm\_08** as processing
environment and the computational region of sentinel band B04 for the
NDVI processing. Then we calculate univariate statistics for the
Sentinel-2A scene. The computed NDVI raster layer will be exported as
geotiff file that can be accessed via an URL.

The following JSON code has 5 process definitions:

 1.  Import of two bands (B04 and B08) of the Sentinel-2A scene
     *S2A\_MSIL2A\_20220420T154941\_N0400\_R054\_T18SUE\_20220421T000632*
 2.  Set the computational region to imported raster layer B04
 3.  Use *r.mapcalc* to compute the NDVI
 4.  Use *r.univar* to compute univariate statistics of the computed NDVI
     raster layer
 5.  Export the computed NDVI as GeoTiff

Run the process chain asynchronously:

In [ ]:
# make a POST request to the actinia data API
request_url = actinia_url + "/locations/nc_spm_08/processing_async_export"
print("actinia POST request:")
print(request_url)
print("---")

process_chain = {
 "list": [{"id": "importer_1",
           "module": "importer",
           "inputs": [{"import_descr": {"source": "S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
                                        "type": "sentinel2",
                                        "sentinel_band": "B04"},
                       "param": "map",
                       "value": "B04"},
                      {"import_descr": {"source": "S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
                                        "type": "sentinel2",
                                        "sentinel_band": "B08"},
                       "param": "map",
                       "value": "B08"}]},
          {"id": "g_region_1",
           "module": "g.region",
           "inputs": [{"param": "raster",
                       "value": "B04"}],
           "flags": "g"},
          {"id": "rmapcalc_1",
           "module": "r.mapcalc",
           "inputs": [{"param": "expression",
                       "value": "NDVI = float(B08 - B04)/(B08 + B04)"}]},
          {"id": "r_univar_sentinel2",
           "module": "r.univar",
           "inputs": [{"param": "map",
                       "value": "NDVI"}],
           "flags": "g"},
          {"id": "exporter_1",
           "module": "exporter",
           "outputs": [{"export": {"type": "raster", "format": "GTiff"},
                        "param": "map",
                        "value": "NDVI"}]}
          ],
 "version": "1"}

request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)
# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

The response requires the polling of the status URL, since the API call
works asynchronously:

In [ ]:
# make a GET request to the actinia data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")

request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

Poll until there is a message **"Processing successfully finished"**

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

The finished response should look similar to this:
```json
 {
   "accept_datetime": "2018-05-02 21:05:34.873031",
   "accept_timestamp": 1525287934.8730297,
   "api_info": {
     "endpoint": "asyncephemeralexportresource",
     "method": "POST",
     "path": "/locations/nc_spm_08/processing_async_export",
     "request_url": "http://actinia.mundialis.de/api/v3/locations/nc_spm_08/processing_async_export"
   },
   "datetime": "2018-05-02 21:09:39.823857",
   "http_code": 200,
   "message": "Processing successfully finished",
   "process_chain_list": [
     {
       "list": [
         {
           "id": "importer_1",
           "inputs": [
             {
               "import_descr": {
                 "sentinel_band": "B04",
                 "source": "S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
                 "type": "sentinel2"
               },
               "param": "map",
               "value": "B04"
             },
             {
               "import_descr": {
                 "sentinel_band": "B08",
                 "source": "S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
                 "type": "sentinel2"
               },
               "param": "map",
               "value": "B08"
             }
           ],
           "module": "importer"
         },
         {
           "flags": "g",
           "id": "g_region_1",
           "inputs": [
             {
               "param": "raster",
               "value": "B04"
             }
           ],
           "module": "g.region"
         },
         {
           "id": "rmapcalc_1",
           "inputs": [
             {
               "param": "expression",
               "value": "NDVI = float(B08 - B04)/(B08 + B04)"
             }
           ],
           "module": "r.mapcalc"
         },
         {
           "flags": "g",
           "id": "r_univar_sentinel2",
           "inputs": [
             {
               "param": "map",
               "value": "NDVI"
             }
           ],
           "module": "r.univar"
         },
         {
           "id": "exporter_1",
           "module": "exporter",
           "outputs": [
             {
               "export": {
                 "format": "GTiff",
                 "type": "raster"
               },
               "param": "map",
               "value": "NDVI"
             }
           ]
         }
       ],
       "version": "1"
     }
   ],
   "process_log": [
     {
      "executable": "i.sentinel.download",
      "id": "i_sentinel_download_S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
      "mapset_size": 421,
      "parameter": [
        "datasource=GCS",
        "query=identifier=S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
        "output=/actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/temp_file_1"
      ],
       "return_code": 0,
       "run_time": 36.222164154052734,
       "stderr": [
         "Downloading data into </actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/temp_file_1>...",
        "Downloading S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632...",
        "...",
        "Downloaded to /actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/temp_file_1/S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632.SAFE",
        ""
       ],
       "stdout": ""
     },
     {
      "executable": "i.sentinel.import",
      "id": "i_sentinel_import_S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632",
      "mapset_size": 376982877,
      "parameter": [
        "input=/actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/temp_file_1",
        "pattern=(B04_10m|B08_10m)",
        "-r"
      ],
      "return_code": 0,
      "run_time": 344.0564560890198,
      "stderr": [
        "Processing <T18SUE_20220420T154941_B08_10m>...",
        "Importing raster map <T18SUE_20220420T154941_B08_10m>...",
        "0..3..6..9..12..15..18..21..24..27..30..33..36..39..42..45..48..51..54..57..60..63..66..69..72..75..78..81..84..87..90..93..96..99..100",
        "Estimated target resolution for input band <T18SUE_20220420T154941_B08_10m>: 9.618686330457596",
        "Using given resolution for input band <T18SUE_20220420T154941_B08_10m>: 10.0",
        "Reprojecting <T18SUE_20220420T154941_B08_10m>...",
        "Rounding to integer after reprojection",
        "Processing <T18SUE_20220420T154941_B04_10m>...",
        "Importing raster map <T18SUE_20220420T154941_B04_10m>...",
        "0..3..6..9..12..15..18..21..24..27..30..33..36..39..42..45..48..51..54..57..60..63..66..69..72..75..78..81..84..87..90..93..96..99..100",
        "Estimated target resolution for input band <T18SUE_20220420T154941_B04_10m>: 9.618686330457596",
        "Using given resolution for input band <T18SUE_20220420T154941_B04_10m>: 10.0",
        "Reprojecting <T18SUE_20220420T154941_B04_10m>...",
        "Rounding to integer after reprojection",
        "Writing metadata to maps...",
        "/usr/local/grass/scripts/i.sentinel.import:830: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.",
        "Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations",
        "  dtype=np.float,",
        "/usr/local/grass/scripts/i.sentinel.import:834: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.",
        "Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations",
        "  dtype=np.float,",
        "/usr/local/grass/scripts/i.sentinel.import:846: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.",
        "Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations",
        "  dtype=np.float,",
        "/usr/local/grass/scripts/i.sentinel.import:850: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.",
        "Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations",
        "  dtype=np.float,",
        ""
      ],
       "stdout": ""
     },
     {
      "executable": "g.rename",
      "id": "rename_S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632_B04",
      "mapset_size": 376982877,
      "parameter": [
        "raster=T18SUE_20220420T154941_B04_10m,B04"
      ],
      "return_code": 0,
      "run_time": 0.15042471885681152,
      "stderr": [
        "Rename raster <T18SUE_20220420T154941_B04_10m> to <B04>",
        ""
      ],
       "stdout": ""
     },
     {
      "executable": "g.rename",
      "id": "rename_S2A_MSIL2A_20220420T154941_N0400_R054_T18SUE_20220421T000632_B08",
      "mapset_size": 376982877,
      "parameter": [
        "raster=T18SUE_20220420T154941_B08_10m,B08"
      ],
      "return_code": 0,
      "run_time": 0.15053868293762207,
      "stderr": [
        "Rename raster <T18SUE_20220420T154941_B08_10m> to <B08>",
        ""
      ],
      "stdout": ""
     },
     {
      "executable": "g.region",
      "id": "g_region_1",
      "mapset_size": 376982871,
      "parameter": [
        "raster=B04",
        "-g"
      ],
      "return_code": 0,
      "run_time": 0.10027408599853516,
      "stderr": [
        ""
      ],
      "stdout": "projection=99\nzone=0\nn=269280\ns=155130\nw=769610\ne=883770\nnsres=10\newres=10\nrows=11415\ncols=11416\ncells=130313640\n"
     },
     {
      "executable": "r.mapcalc",
      "id": "rmapcalc_1",
      "mapset_size": 808176157,
      "parameter": [
        "expression=NDVI = float(B08 - B04)/(B08 + B04)"
      ],
      "return_code": 0,
      "run_time": 25.457623958587646,
      "stderr": [
        ""
      ],
      "stdout": ""
     },
     {
      "executable": "r.univar",
      "id": "r_univar_sentinel2",
      "mapset_size": 808176157,
      "parameter": [
        "map=NDVI",
        "-g"
      ],
      "return_code": 0,
      "run_time": 5.166007995605469,
      "stderr": [
        ""
      ],
      "stdout": "n=120547065\nnull_cells=9766575\ncells=130313640\nmin=-0.468370884656906\nmax=0.720661163330078\nrange=1.18903204798698\nmean=0.25310113195823\nmean_of_abs=0.283006503246459\nstddev=0.219731453415328\nvariance=0.0482819116200123\ncoeff_var=86.8156739226241\nsum=30510598.6057423\n"
     },
     {
      "executable": "r.out.gdal",
      "id": "exporter_raster_NDVI",
      "mapset_size": 808176157,
      "parameter": [
        "-fmt",
        "input=NDVI",
        "format=GTiff",
        "output=/actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/NDVI.tif",
        "overviews=5",
        "createopt=BIGTIFF=YES,COMPRESS=LZW,TILED=YES"
      ],
      "return_code": 0,
      "run_time": 20.729568481445312,
      "stderr": [
        "Checking GDAL data type and nodata value...",
        "2..5..8..11..14..17..20..23..26..29..32..35..38..41..44..47..50..53..56..59..62..65..68..71..74..77..80..83..86..89..92..95..98..100",
        "Using GDAL data type <Float32>",
        "Input raster map contains cells with NULL-value (no-data). The value -nan will be used to represent no-data values in the input map. You can specify a nodata value with the nodata option.",
        "Exporting raster data to GTiff format...",
        "ERROR 6: /actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/NDVI.tif, band 1: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.",
        "2..5..8..11..14..17..20..23..26..29..32..35..38..41..44..47..50..53..56..59..62..65..68..71..74..77..80..83..86..89..92..95..98..100",
        "Building overviews ...",
        "r.out.gdal complete. File </actinia_core/workspace/temp_db/gisdbase_dbbd7bc372e942d0953fa0f7d019cacb/.tmp/NDVI.tif> created.",
        ""
      ],
      "stdout": ""
     }
   ],
   "process_results": {},
   "progress": {
     "num_of_steps": 8,
     "step": 8
   },
   "resource_id": "resource_id-60f3f012-4220-46ec-9110-694df49006c4",
   "status": "finished",
   "time_delta": 440.42616844177246,
   "timestamp": 1659017682.0043712,
   "urls": {
     "resources": [
       "http://actinia.mundialis.de/api/v3/resource/demouser/resource_id-60f3f012-4220-46ec-9110-694df49006c4/NDVI.tiff"
     ],
     "status": "http://actinia.mundialis.de/api/v3/resources/demouser/resource_id-60f3f012-4220-46ec-9110-694df49006c4"
   },
   "user_id": "demouser"
 }
```

In [ ]:
# print formatted JSON
print_as_json(jsonResponse)